# Поиск каверов композиций
<br>Разработка ML-модели для сопоставления текстов музыкальных произведений и для поиска каверов (вариации обработки оригинала с элементами новой аранжировки) по их текстам

<br>**Описание проекта**
<br>Обнаружение треков каверов - важная продуктовая задача, которая может значительно улучшить качество рекомендаций музыкального сервиса и повысить счастье наших пользователей. 
<br>Если мы умеем с высокой точностью классифицировать каверы и связывать их между собой, то можно предложить пользователю новые возможности для управления потоком треков. 
<br>Например:
- по желанию пользователя можем полностью исключить каверы из рекомендаций;
- показать все каверы на любимый трек пользователя;
- контролировать долю каверов в ленте пользователя.

<br>**Цель проекта** 
<br>Необходимо разработать ML-продукт, который:
- классифицирует треки по признаку кавер-некавер;
- связывает (группирует) каверы и исходный трек;
- находит исходный трек в цепочке каверов.
 
<br>**Описание данных**
- Файл covers.json содержит разметку каверов, сделанную редакторами сервиса:
    - track_id - уникальный идентификатор трека;
    - track_remake_type - метка, присвоенная редакторами. Может принимать значения ORIGINAL и COVER;
    - original_track_id - уникальный идентификатор исходного трека.
- Метаинформация:
    - track_id - уникальный идентификатор трека;
    - dttm - первая дата появления информации о треке;
    - title - название трека;
    - language - язык исполнения;
    - isrc - международный уникальный идентификатор трека;
    - genres - жанры;
    - duration - длительность трека.
- Текст песен:
    - track_id - уникальный идентификатор трека;
    - lyricId - уникальный идентификатор текста;
    - text - текст трека.
 
<br>**Целевой метрикой выбрана** `f1`

## Импорт библиотек

In [41]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
#sklearn, CatBoost, XGBoost, LightGBM, NLP

## EDA

### Загрузка данных

In [3]:
try:
    covers = pd.read_json('data/covers.json', orient='records', lines=True)
    lyrics = pd.read_json('data/lyrics.json', orient='records', lines=True)
    meta = pd.read_json('data/meta.json', orient='records', lines=True)
    print('загружено из каталога "data"')
except:
    covers = pd.read_json('/content/covers.json', orient='records', lines=True)
    lyrics = pd.read_json('/content/lyrics.json', orient='records', lines=True)
    meta = pd.read_json('/content/meta.json', orient='records', lines=True)
    print('загружено из каталога "content"')

np.array(covers).shape, np.array(lyrics).shape, np.array(meta).shape

загружено из каталога "data"


((71597, 3), (11414, 3), (71769, 7))

### Датафрейм `lyrics`

In [31]:
lyrics.info()
display(lyrics.sample(3))
print('явные повторы строк:', lyrics.duplicated().sum())
display('количество уникальных записей:', lyrics.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11414 entries, 0 to 11413
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   lyricId   11414 non-null  object
 1   text      11414 non-null  object
 2   track_id  11414 non-null  object
dtypes: object(3)
memory usage: 267.6+ KB


,lyricId,text,track_id
6240,64d2cb9e6f771a261fe3e6d4d728d629,Time goes by so slowly\nTime goes by so slowly...,417ff9fb2829051aad4f66197f3f4b76
7891,15a916d5cd7ee155a386fd79be2fcf7e,If you wanna run away with me\nI know a galaxy...,09851900f74faae97d3c08586df2fb2d
10776,a3015b4681996bd76fe9f2436c1913dd,I Just Can't Help Thinking\nWhat You've Done T...,f01a22270b4545443b2133db783d3a23


явные повторы строк: 0


'количество уникальных записей:'

lyricId     10915
text        10644
track_id    10277
dtype: int64

Выводы:
- тексты представлены на различных языках (английский, русский, испанский)
- явных повторов строк не обнаружено, при этом количество уникальных ID текстов и треков меньше,
  <br>чем всего строк, что означает использование текстов в других треках

### Датафрейм `meta`

In [40]:
meta.info()
display(meta.sample()) #genres - list
print('явные повторы строк:', meta[['track_id', 'dttm', 'title', 'language', 'isrc', 'duration']].duplicated().sum())
display('количество уникальных записей:', meta[['track_id', 'dttm', 'title', 'language', 'isrc', 'duration']].nunique())
display('список представленных стран:', meta['language'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71769 entries, 0 to 71768
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   track_id  71768 non-null  object 
 1   dttm      71768 non-null  float64
 2   title     71768 non-null  object 
 3   language  21969 non-null  object 
 4   isrc      71455 non-null  object 
 5   genres    71768 non-null  object 
 6   duration  71768 non-null  float64
dtypes: float64(2), object(5)
memory usage: 3.8+ MB


,track_id,dttm,title,language,isrc,genres,duration
64443,ed9d3c601d04ab5aa63de2fc4fd76d99,1.616545e+12,Volta pro Seu Nego,None,QZGLS2122461,"[FOLK, LATINFOLK]",140820.0


явные повторы строк: 0


'количество уникальных записей:'

track_id    71768
dttm        27726
title       45462
language       85
isrc        71283
duration    23597
dtype: int64

'список представленных стран:'

array(['EN', None, 'ES', 'HI', 'DE', 'RU', 'TR', 'HU', 'TH', 'PL', 'FR',
       'NY', 'AF', 'AS', 'UZ', 'HT', 'EL', 'AZ', 'IT', 'PA', 'PT', 'TA',
       'JA', 'ML', 'VI', 'ID', 'LA', 'CS', 'SI', 'UK', 'OR', 'HR', 'AR',
       'KK', 'FI', 'IE', 'ZH', 'AB', 'KN', 'FA', 'BN', 'TL', 'SK', 'KS',
       'SV', 'RO', 'TN', 'KO', 'MS', 'BM', 'HY', 'TW', 'MY', 'CA', 'NL',
       'ET', 'TE', 'MN', 'HE', 'SQ', 'IG', 'MR', 'BE', 'LT', 'UR', 'IA',
       'GN', 'SW', 'NO', 'GU', 'KY', 'KU', 'IS', 'TG', 'SR', 'DA', 'LO',
       'LV', 'SE', 'WO', 'SA', 'YO', 'ST', 'HA', 'AV', 'IU'], dtype=object)

In [32]:
# from datetime import datetime

# unix_timestamp1 = 1.555760e+12 
# unix_timestamp2 = 1.626110e+12 

# # Преобразование временной метки в объект datetime
# dt1 = datetime.fromtimestamp(unix_timestamp1 / 1000) #2019-04-20 14:33:20
# dt2 = datetime.fromtimestamp(unix_timestamp2 / 1000) #2021-07-12 20:13:20

# print(dt1 < dt2) #True
# unix_timestamp1 < unix_timestamp2 #True

Выводы:
- явных повторов строк не обнаружено
- количество стран в колонке `language` - 85
- информация о странах внесена менее, чем в 30% записей
- все `track_id` уникальны
- даты представлены в unix формате
- можно удалить признаки:
  - `duration` - не несёт важной информации, т.к. продолжительность кавера может отличаться от исходника
  - `genres` - не несёт важной информации, т.к. жанр кавера может отличаться от исходника
  - `isrc` - не несёт важной информации, т.к. имеется признак `track_id`, а ещё в нём есть пропуски

### Датафрейм `covers`

In [38]:
covers.info()
display(covers.sample())
print('явные повторы строк:', covers.duplicated().sum())
display('количество уникальных записей:', covers.nunique())
print('оригинальных треков указано:', round(covers['original_track_id'].count() / covers['track_id'].count() * 100, 2), '% записей')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71597 entries, 0 to 71596
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_track_id  4821 non-null   object
 1   track_id           71597 non-null  object
 2   track_remake_type  71597 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB


,original_track_id,track_id,track_remake_type
10203,None,8c5b58a406d3854f860b5a1a6692dca0,COVER


явные повторы строк: 0


'количество уникальных записей:'

original_track_id     4468
track_id             71597
track_remake_type        2
dtype: int64

оригинальных треков указано: 6.73 % записей


Выводы:
- явных повторов строк не обнаружено
- количество указанных оригинальных треков - 6.73 % от общего числа треков